# Course Work 
### * By: Csc/19D/4120

### Instructions: * Using these AI chat tools namely ChatGPT, Perplexity AI and BlackBox AI demonstrate how prompts can be used to design, develop and -deploy a multimedia app on a server.
Submit both the prompts and the results in chronological order.

### Promt 1
 please summarise the step to deploy a multimedia app on a server


# Response 1:


To deploy a multimedia app on a server, ensure your application is ready and working locally with all dependencies listed in package.json. Choose a hosting provider like Heroku, AWS, Vercel, or DigitalOcean, and create an account. Install the CLI tools for your chosen provider and initialize a Git repository for your project. Create a new app on the hosting provider (e.g., heroku create your-app-name), configure deployment settings, and push your code to the remote repository (git push heroku main). After deployment, verify the app is running correctly, configure necessary databases and storage solutions, and set up monitoring tools for maintenance and performance tracking. For Heroku, use commands like heroku login, heroku create, and heroku open to manage the deployment process.

### Design 1.1 

Develop a multimedia application that enables users to upload, view, like, share, and comment on videos.


## Step-by-Step Development Guide
1. Set Up the Backend (Node.js with Express.js)
Initialize the Project and Install Dependencies:

In [ ]:
mkdir multimedia-app
cd multimedia-app
npm init -y
npm install express mongoose multer bcryptjs jsonwebtoken cors


Create the Server:
Create a file named server.js and add the following code:

In [ ]:
const express = require('express');
const mongoose = require('mongoose');
const multer = require('multer');
const cors = require('cors');
const app = express();
const PORT = process.env.PORT || 5000;

// Middleware
app.use(cors());
app.use(express.json());
app.use(express.urlencoded({ extended: true }));

// MongoDB connection
mongoose.connect('mongodb://localhost/multimedia-app', { useNewUrlParser: true, useUnifiedTopology: true })
  .then(() => console.log('MongoDB connected'))
  .catch(err => console.error(err));

// Define storage for multer
const storage = multer.diskStorage({
  destination: 'uploads/',
  filename: (req, file, cb) => {
    cb(null, Date.now() + '-' + file.originalname);
  }
});

const upload = multer({ storage: storage });

// Models
const Video = mongoose.model('Video', new mongoose.Schema({
  title: String,
  description: String,
  fileUrl: String,
  likes: { type: Number, default: 0 },
  comments: [{ user: String, comment: String }],
  createdAt: { type: Date, default: Date.now }
}));

// Routes
app.post('/upload', upload.single('video'), async (req, res) => {
  const { title, description } = req.body;
  const fileUrl = `uploads/${req.file.filename}`;
  const newVideo = new Video({ title, description, fileUrl });
  await newVideo.save();
  res.json(newVideo);
});

app.get('/videos', async (req, res) => {
  const videos = await Video.find().sort({ createdAt: -1 });
  res.json(videos);
});

app.get('/videos/:id', async (req, res) => {
  const video = await Video.findById(req.params.id);
  res.json(video);
});

app.post('/like/:id', async (req, res) => {
  const video = await Video.findById(req.params.id);
  video.likes += 1;
  await video.save();
  res.json(video);
});

app.post('/comment/:id', async (req, res) => {
  const { user, comment } = req.body;
  const video = await Video.findById(req.params.id);
  video.comments.push({ user, comment });
  await video.save();
  res.json(video);
});

app.use('/uploads', express.static('uploads'));

app.listen(PORT, () => {
  console.log(`Server running on http://localhost:${PORT}`);
});


2. Set Up the Frontend (Bootstrap)
Create the App:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Multimedia App</title>
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
  <script defer src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
  <script defer src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.5.4/dist/umd/popper.min.js"></script>
  <script defer src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
  <script defer src="https://cdn.jsdelivr.net/npm/axios/dist/axios.min.js"></script>
  <script defer src="index.js"></script>
</head>
<body>
  <nav class="navbar navbar-expand-lg navbar-light bg-light">
    <a class="navbar-brand" href="#">Multimedia App</a>
    <div class="collapse navbar-collapse" id="navbarNav">
      <ul class="navbar-nav ml-auto">
        <li class="nav-item">
          <a class="nav-link" href="#" id="uploadLink">Upload</a>
        </li>
      </ul>
    </div>
  </nav>
  <div class="container mt-4" id="content">
    <!-- Content will be loaded dynamically here -->
  </div>
</body>
</html>


In [ ]:
document.addEventListener("DOMContentLoaded", () => {
  loadHome();

  document.getElementById('uploadLink').addEventListener('click', (e) => {
    e.preventDefault();
    loadUploadForm();
  });
});

function loadHome() {
  axios.get('http://localhost:5000/videos')
    .then(response => {
      const videos = response.data;
      const content = document.getElementById('content');
      content.innerHTML = `
        <h1>All Videos</h1>
        <div class="row">
          ${videos.map(video => `
            <div class="col-md-4">
              <div class="card mb-4">
                <video controls class="card-img-top">
                  <source src="http://localhost:5000/${video.fileUrl}" type="video/mp4">
                </video>
                <div class="card-body">
                  <h5 class="card-title">${video.title}</h5>
                  <p class="card-text">${video.description}</p>
                  <button class="btn btn-primary" onclick="loadVideoDetail('${video._id}')">View Details</button>
                </div>
              </div>
            </div>
          `).join('')}
        </div>
      `;
    })
    .catch(error => console.error(error));
}

function loadUploadForm() {
  const content = document.getElementById('content');
  content.innerHTML = `
    <h1>Upload Video</h1>
    <form id="uploadForm">
      <div class="form-group">
        <label for="title">Title</label>
        <input type="text" class="form-control" id="title" required>
      </div>
      <div class="form-group">
        <label for="description">Description</label>
        <textarea class="form-control" id="description" required></textarea>
      </div>
      <div class="form-group">
        <label for="video">Video</label>
        <input type="file" class="form-control" id="video" required>
      </div>
      <button type="submit" class="btn btn-primary">Upload</button>
    </form>
  `;

  document.getElementById('uploadForm').addEventListener('submit', (e) => {
    e.preventDefault();
    const formData = new FormData();
    formData.append('video', document.getElementById('video').files[0]);
    formData.append('title', document.getElementById('title').value);
    formData.append('description', document.getElementById('description').value);

    axios.post('http://localhost:5000/upload', formData)
      .then(response => {
        alert('Video uploaded successfully!');
        loadHome();
      })
      .catch(error => {
        console.error(error);
        alert('Failed to upload video.');
      });
  });
}

function loadVideoDetail(id) {
  axios.get(`http://localhost:5000/videos/${id}`)
    .then(response => {
      const video = response.data;
      const content = document.getElementById('content');
      content.innerHTML = `
        <h1>${video.title}</h1>
        <video controls class="w-100">
          <source src="http://localhost:5000/${video.fileUrl}" type="video/mp4">
        </video>
        <div class="mt-3">
          <button class="btn btn-primary" onclick="likeVideo('${video._id}')">Like (${video.likes})</button>
        </div>
        <div class="mt-3">
          <h3>Comments</h3>
          <form id="commentForm">
            <div class="form-group">
              <input type="text" class="form-control" id="user" placeholder="Your name" required>
            </div>
            <div class="form-group">
              <textarea class="form-control" id="comment" placeholder="Your comment" required></textarea>
            </div>
            <button type="submit" class="btn btn-primary">Submit</button>
          </form>
          <ul class="list-group mt-3">
            ${video.comments.map(comment => `
              <li class="list-group-item">
                <strong>${comment.user}:</strong> ${comment.comment}
              </li>
            `).join('')}
          </ul>
        </div>
      `;

      document.getElementById('commentForm').addEventListener('submit', (e) => {
        e.preventDefault();
        const user = document.getElementById('user').value;
        const comment = document.getElementById('comment').value;

        axios.post(`http://localhost:5000/comment/${id}`, { user, comment })
          .then(response => {
            loadVideoDetail(id);
          })
          .catch(error => {
            console.error(error);
            alert('Failed to submit comment.');
          });
      });
    })
    .catch(error => console.error(error));
}

function likeVideo(id) {
  axios.post(`http://localhost:5000/like/${id}`)
    .then(response => {
      loadVideoDetail(id);
    })
    .catch(error => console.error(error));
}


In [ ]:
# hosted link

